In [1]:
import pandas as pd
import numpy as np
from utils import findall
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Input, Concatenate, BatchNormalization, Add
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.models import Model
from keras.preprocessing.text import one_hot
import tensorflow as tf
from keras import backend as K
from keras.engine.topology import Layer

/home/stein/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
mols = pd.read_csv("data/original_data/df_molecule.csv")
affs = pd.read_csv("data/original_data/df_affinity.csv")
proteins = pd.read_csv("data/original_data/df_protein.csv")

In [3]:
#(affs["Ki"] == np.float("inf")).sum()

In [4]:
mols

,smiles,name,cyp_2c9,half-life,ames_toxicity,logBB,cyp_3a4,cell_permeability,logP,fathead_minnow_toxicity,...,cyp_2d6,tetrahymena_pyriformis_toxicity,Vdd,honey_bee,p_glycoprotein_inhibition,NOAEL,solubility,logS,bbb,Molecule_ID
0,COc1ccccc1CN[C@H]1CCCN[C@H]1C(c1ccccc1)c1ccccc1,9,0.742983,16.663698,0.623255,0.453208,0.311594,4.919215,5.587535,0.005905,...,0.363031,3.740105,24.211564,0.800019,0.260743,-0.757354,-5.401525,-5.249560,0.087650,0
1,CCN(CCN1CCCC1)CCc1ccc(Cl)c(Cl)c1,12,0.854373,25.629075,0.624465,0.933780,0.117451,4.723896,4.318473,0.034951,...,0.280685,2.174478,27.109723,0.846931,0.511252,-1.141304,-5.089023,-4.562299,0.000608,1
2,CN(CCN1CCCCCC1)CCc1ccc(Cl)c(Cl)c1,13,0.853763,18.774007,0.745154,0.898950,0.102385,4.731113,4.462095,0.030430,...,0.197318,2.038386,20.560356,0.854921,0.439826,-1.163596,-5.124717,-4.754227,0.000570,2
3,C[C@@H](CN1CCCC1)N(C)CCc1ccc(Cl)c(Cl)c1,14,0.855956,15.450438,0.687151,0.868897,0.315907,4.470623,4.269530,0.033857,...,0.376436,2.082341,14.791946,0.848883,0.604665,-1.155990,-4.973178,-4.391506,0.029107,3
4,CN(CCN1CCCCC1)CCc1ccc(Cl)c(Cl)c1,15,0.844784,20.252838,0.690372,0.887504,0.106763,4.723733,4.193617,0.036015,...,0.207776,1.895097,22.261876,0.859958,0.494917,-1.158525,-4.886913,-4.460482,0.000603,4
5,CCN(CC)CCN(C)CCc1ccc(Cl)c(Cl)c1,17,0.818163,65.471019,0.262465,0.980559,0.240993,4.611797,4.358286,0.039509,...,0.459334,2.304098,54.651727,0.841768,0.727344,-1.102987,-5.718917,-4.748726,0.007349,5
6,CN(C)CCN(C)CCc1ccc(Cl)c(Cl)c1,18,0.767587,54.779876,0.399165,0.826934,0.145238,4.580892,3.620532,0.063525,...,0.407507,1.529833,46.740471,0.797694,0.789445,-1.193677,-4.883039,-3.892498,0.007610,6
7,CN(CCN1CCCC1)CCc1ccccc1,19,0.827739,10.836824,0.944508,0.828867,0.645086,4.764075,2.812568,0.057156,...,0.584620,1.402493,14.462664,0.858086,0.636018,-1.152324,-3.787272,-2.765288,0.000504,7
8,Clc1ccc(CCN(CCN2CCCC2)CC=C)cc1Cl,20,0.847625,19.503976,0.723756,0.766071,0.550406,4.827510,3.985283,0.025073,...,0.566617,2.593200,24.114881,0.608626,0.395796,-1.036255,-5.306387,-4.686547,0.000427,8
9,CN(C)CCNCCc1ccc(Cl)c(Cl)c1,21,0.745367,60.438707,0.372340,0.727982,0.232483,4.637589,3.484027,0.067992,...,0.326004,1.463334,59.391358,0.826168,0.816970,-1.252259,-4.575261,-3.739378,0.077738,9


In [5]:
affs.merge(mols, on=["Molecule_ID"])

,Seq_ID,Molecule_ID,Ki,smiles,name,cyp_2c9,half-life,ames_toxicity,logBB,cyp_3a4,...,biodegradation,cyp_2d6,tetrahymena_pyriformis_toxicity,Vdd,honey_bee,p_glycoprotein_inhibition,NOAEL,solubility,logS,bbb
0,523,41360,16.000000,O=C(N[C@H]1CN2CCC1CC2)c1cccc2ccccc12,281605,0.755509,11.923168,0.511163,0.140675,0.772978,...,0.036610,0.521963,2.390618,7.920029,0.945635,0.802797,-0.789534,-4.149490,-3.774362,0.000809
1,1313,55359,15.987163,Oc1ccc(Br)cc1CN(C(=O)Nc1ccccc1)c1ccc(Cl)cc1,409833,0.587683,12.592266,0.749026,-0.296284,0.487686,...,0.012717,0.957537,3.450799,16.216753,0.734741,0.701690,-0.702192,-6.478204,-6.345670,0.001147
2,1313,55310,15.806875,CC(Cn1c(C)ncc1[N+]([O-])=O)OC(=O)\C=C\c1ccc(cc...,409501,0.653073,6.923901,0.687132,-0.864827,0.295937,...,0.023377,0.920578,3.693061,7.208750,0.298581,0.721135,-0.630267,-6.066857,-5.487503,0.002551
3,1313,55357,15.793174,Oc1ccc(Br)cc1CN(C(=O)Nc1ccccc1)c1ccc(F)cc1,409831,0.588648,8.640816,0.710999,-0.063166,0.651343,...,0.008780,0.927052,3.363629,13.563803,0.817710,0.692195,-0.599661,-6.052654,-6.141785,0.001064
4,80,957,15.522879,CCCCC(NC(=O)[C@@H](NC(=O)[C@@H](NC(=O)OCc1cccc...,6717,0.827357,2.629953,0.986039,-0.631213,0.403351,...,0.347914,0.934438,3.999235,5.187924,0.435249,0.767342,-0.184689,-4.182205,-4.125540,0.678658
5,1313,55361,15.261219,Oc1ccc(Br)cc1CN(C(=O)Nc1ccccc1)c1ccc(Br)cc1,409835,0.600398,7.417559,0.732160,-0.307169,0.649581,...,0.018465,0.940413,3.454977,12.084018,0.729295,0.698355,-0.673126,-6.431789,-6.345439,0.001250
6,44,74385,15.221848,CC(C)CN(C[C@@H](O)[C@H](Cc1ccc(OCc2ccccn2)cc1)...,578803,0.851588,8.766241,0.826906,-0.911199,0.100540,...,0.003902,0.546829,5.896251,11.208522,0.372485,0.066304,-0.447146,-4.820603,-5.180406,0.973049
7,1313,55360,15.151196,Oc1ccc(Br)cc1CN(C(=O)Nc1ccccc1)c1ccccc1Br,409834,0.597863,7.764490,0.665680,-0.235095,0.629789,...,0.017900,0.945098,3.428534,12.286972,0.715259,0.683242,-0.688133,-6.096173,-6.171887,0.000895
8,1313,55365,15.146911,Oc1c(Br)cc(Br)cc1CN(C(=O)Nc1ccccc1)c1ccc(F)cc1,409839,0.601831,9.189799,0.745176,-0.025883,0.626886,...,0.004404,0.949439,3.598352,15.449378,0.824807,0.657543,-0.584871,-6.580716,-6.601081,0.001571
9,1313,55364,15.096910,Oc1c(Br)cc(Br)cc1CN(C(=O)Nc1ccccc1)c1ccccc1F,409838,0.599585,9.397911,0.706848,0.034426,0.621534,...,0.004298,0.950959,3.578400,15.845965,0.810704,0.649746,-0.605630,-6.234195,-6.453171,0.001238


In [6]:
proteins

,Protein_ID,Sequence
0,0,MDNVLPVDSDLSPNISTNTSEPNQFVQPAWQIVLWAAAYTVIVVTS...
1,1,MQWAVGRRWAWAALLLAVAAVLTQVVWLWLGTQSFVFQREEIAQLA...
2,2,MDPLNLSWYDDDLERQNWSRPFNGSEGKADRPHYNYYAMLLTLLIF...
3,3,MDSSTGPGNTSDCSDPLAQASCSPAPGSWLNLSHVDGNQSDPCGLN...
4,4,MEPVPSARAELQFSLLANVSDTFPSAFPSASANASGSPGARSASSL...
5,5,MEPAPSAGAELQPPLFANASDAYPSACPSAGANASGPPGARSASSL...
6,6,MGRRRQGPAQPASELPARNACLLPNGSAWLPGWAEPDGNGSAGPQD...
7,7,MDSPIQIFRGEPGPTCAPSACLPPNSSAWFPGWAEPDSNGSAGSED...
8,8,MELNSSSRVDSEFRYTLFPIVYSIIFVLGIIANGYVLWVFARLYPS...
9,9,MIWGVLMMGILLPQCSAHPGFFTSIGQMTDLIHNEKDLVTSLKDYI...


In [7]:
proteins["Sequence"][0]

'MDNVLPVDSDLSPNISTNTSEPNQFVQPAWQIVLWAAAYTVIVVTSVVGNVVVMWIILAHKRMRTVTNYFLVNLAFAEASMAAFNTVVNFTYAVHNEWYYGLFYCKFHNFFPIAAVFASIYSMTAVAFDRYMAIIHPLQPRLSATATKVVICVIWVLALLLAFPQGYYSTTETMPSRVVCMIEWPEHPNKIYEKVYHICVTVLIYFLPLLVIGYAYTVVGITLWASEIPGDSSDRYHEQVSAKRKVVKMMIVVVCTFAICWLPFHIFFLLPYINPDLYLKKFIQQVYLAIMWLAMSSTMYNPIIYCCLNDRFRLGFKHAFRCCPFISAGDYEGLEMKSTRYLQTQGSVYKVSRLETTISTVVGAHEEEPEDGPKATPSSLDLTSNCSSRSDSKTMTESFSFSSNVLS'

In [8]:
texts_protein = list(proteins["Sequence"].apply(lambda x: findall(x.upper(), 3, 3)))

protein_corpus = list(map(lambda x: " ".join(i for i in x), texts_protein))

In [9]:
kk = np.array(mols["smiles"].apply(lambda x: len(x)))

In [10]:
(kk<100).sum()

105044

In [11]:
mols

,smiles,name,cyp_2c9,half-life,ames_toxicity,logBB,cyp_3a4,cell_permeability,logP,fathead_minnow_toxicity,...,cyp_2d6,tetrahymena_pyriformis_toxicity,Vdd,honey_bee,p_glycoprotein_inhibition,NOAEL,solubility,logS,bbb,Molecule_ID
0,COc1ccccc1CN[C@H]1CCCN[C@H]1C(c1ccccc1)c1ccccc1,9,0.742983,16.663698,0.623255,0.453208,0.311594,4.919215,5.587535,0.005905,...,0.363031,3.740105,24.211564,0.800019,0.260743,-0.757354,-5.401525,-5.249560,0.087650,0
1,CCN(CCN1CCCC1)CCc1ccc(Cl)c(Cl)c1,12,0.854373,25.629075,0.624465,0.933780,0.117451,4.723896,4.318473,0.034951,...,0.280685,2.174478,27.109723,0.846931,0.511252,-1.141304,-5.089023,-4.562299,0.000608,1
2,CN(CCN1CCCCCC1)CCc1ccc(Cl)c(Cl)c1,13,0.853763,18.774007,0.745154,0.898950,0.102385,4.731113,4.462095,0.030430,...,0.197318,2.038386,20.560356,0.854921,0.439826,-1.163596,-5.124717,-4.754227,0.000570,2
3,C[C@@H](CN1CCCC1)N(C)CCc1ccc(Cl)c(Cl)c1,14,0.855956,15.450438,0.687151,0.868897,0.315907,4.470623,4.269530,0.033857,...,0.376436,2.082341,14.791946,0.848883,0.604665,-1.155990,-4.973178,-4.391506,0.029107,3
4,CN(CCN1CCCCC1)CCc1ccc(Cl)c(Cl)c1,15,0.844784,20.252838,0.690372,0.887504,0.106763,4.723733,4.193617,0.036015,...,0.207776,1.895097,22.261876,0.859958,0.494917,-1.158525,-4.886913,-4.460482,0.000603,4
5,CCN(CC)CCN(C)CCc1ccc(Cl)c(Cl)c1,17,0.818163,65.471019,0.262465,0.980559,0.240993,4.611797,4.358286,0.039509,...,0.459334,2.304098,54.651727,0.841768,0.727344,-1.102987,-5.718917,-4.748726,0.007349,5
6,CN(C)CCN(C)CCc1ccc(Cl)c(Cl)c1,18,0.767587,54.779876,0.399165,0.826934,0.145238,4.580892,3.620532,0.063525,...,0.407507,1.529833,46.740471,0.797694,0.789445,-1.193677,-4.883039,-3.892498,0.007610,6
7,CN(CCN1CCCC1)CCc1ccccc1,19,0.827739,10.836824,0.944508,0.828867,0.645086,4.764075,2.812568,0.057156,...,0.584620,1.402493,14.462664,0.858086,0.636018,-1.152324,-3.787272,-2.765288,0.000504,7
8,Clc1ccc(CCN(CCN2CCCC2)CC=C)cc1Cl,20,0.847625,19.503976,0.723756,0.766071,0.550406,4.827510,3.985283,0.025073,...,0.566617,2.593200,24.114881,0.608626,0.395796,-1.036255,-5.306387,-4.686547,0.000427,8
9,CN(C)CCNCCc1ccc(Cl)c(Cl)c1,21,0.745367,60.438707,0.372340,0.727982,0.232483,4.637589,3.484027,0.067992,...,0.326004,1.463334,59.391358,0.826168,0.816970,-1.252259,-4.575261,-3.739378,0.077738,9


In [12]:
texts_smiles = list(mols["smiles"].apply(lambda x: findall(x.upper(), 1, 1)))
smile_corpus = list(map(lambda x: " ".join(i for i in x), texts_smiles))

In [13]:
mols["smile_corpus"] = smile_corpus

In [14]:
proteins["protein_corpus"] = protein_corpus

In [15]:
proteins

,Protein_ID,Sequence,protein_corpus
0,0,MDNVLPVDSDLSPNISTNTSEPNQFVQPAWQIVLWAAAYTVIVVTS...,MDN VLP VDS DLS PNI STN TSE PNQ FVQ PAW QIV LW...
1,1,MQWAVGRRWAWAALLLAVAAVLTQVVWLWLGTQSFVFQREEIAQLA...,MQW AVG RRW AWA ALL LAV AAV LTQ VVW LWL GTQ SF...
2,2,MDPLNLSWYDDDLERQNWSRPFNGSEGKADRPHYNYYAMLLTLLIF...,MDP LNL SWY DDD LER QNW SRP FNG SEG KAD RPH YN...
3,3,MDSSTGPGNTSDCSDPLAQASCSPAPGSWLNLSHVDGNQSDPCGLN...,MDS STG PGN TSD CSD PLA QAS CSP APG SWL NLS HV...
4,4,MEPVPSARAELQFSLLANVSDTFPSAFPSASANASGSPGARSASSL...,MEP VPS ARA ELQ FSL LAN VSD TFP SAF PSA SAN AS...
5,5,MEPAPSAGAELQPPLFANASDAYPSACPSAGANASGPPGARSASSL...,MEP APS AGA ELQ PPL FAN ASD AYP SAC PSA GAN AS...
6,6,MGRRRQGPAQPASELPARNACLLPNGSAWLPGWAEPDGNGSAGPQD...,MGR RRQ GPA QPA SEL PAR NAC LLP NGS AWL PGW AE...
7,7,MDSPIQIFRGEPGPTCAPSACLPPNSSAWFPGWAEPDSNGSAGSED...,MDS PIQ IFR GEP GPT CAP SAC LPP NSS AWF PGW AE...
8,8,MELNSSSRVDSEFRYTLFPIVYSIIFVLGIIANGYVLWVFARLYPS...,MEL NSS SRV DSE FRY TLF PIV YSI IFV LGI IAN GY...
9,9,MIWGVLMMGILLPQCSAHPGFFTSIGQMTDLIHNEKDLVTSLKDYI...,MIW GVL MMG ILL PQC SAH PGF FTS IGQ MTD LIH NE...


In [16]:
smile_corpus[0]

'C O C 1 C C C C C 1 C N [ C @ H ] 1 C C C N [ C @ H ] 1 C ( C 1 C C C C C 1 ) C 1 C C C C C 1'

In [17]:
smile_corpus[1]

'C C N ( C C N 1 C C C C 1 ) C C C 1 C C C ( C L ) C ( C L ) C 1'

In [18]:
smile_corpus[0]

'C O C 1 C C C C C 1 C N [ C @ H ] 1 C C C N [ C @ H ] 1 C ( C 1 C C C C C 1 ) C 1 C C C C C 1'

In [19]:
all_smiles = mols["smiles"].values

In [20]:
tokens = set()
for s in all_smiles:
    tokens = tokens.union(set(c for c in s))
tokens = sorted(list(tokens))

In [21]:
len(tokens)

54

In [22]:
mol_vocab_size = 60

In [23]:
mol_encoded_docs = [one_hot(d, mol_vocab_size) for d in smile_corpus]

In [24]:
mol_encoded_docs[0]

[2,
 7,
 2,
 48,
 2,
 2,
 2,
 2,
 2,
 48,
 2,
 15,
 2,
 44,
 48,
 2,
 2,
 2,
 15,
 2,
 44,
 48,
 2,
 2,
 48,
 2,
 2,
 2,
 2,
 2,
 48,
 2,
 48,
 2,
 2,
 2,
 2,
 2,
 48]

In [25]:
mol_encoded_docs

[[2,
  7,
  2,
  48,
  2,
  2,
  2,
  2,
  2,
  48,
  2,
  15,
  2,
  44,
  48,
  2,
  2,
  2,
  15,
  2,
  44,
  48,
  2,
  2,
  48,
  2,
  2,
  2,
  2,
  2,
  48,
  2,
  48,
  2,
  2,
  2,
  2,
  2,
  48],
 [2,
  2,
  15,
  2,
  2,
  15,
  48,
  2,
  2,
  2,
  2,
  48,
  2,
  2,
  2,
  48,
  2,
  2,
  2,
  2,
  30,
  2,
  2,
  30,
  2,
  48],
 [2,
  15,
  2,
  2,
  15,
  48,
  2,
  2,
  2,
  2,
  2,
  2,
  48,
  2,
  2,
  2,
  48,
  2,
  2,
  2,
  2,
  30,
  2,
  2,
  30,
  2,
  48],
 [2,
  2,
  44,
  2,
  15,
  48,
  2,
  2,
  2,
  2,
  48,
  15,
  2,
  2,
  2,
  2,
  48,
  2,
  2,
  2,
  2,
  30,
  2,
  2,
  30,
  2,
  48],
 [2,
  15,
  2,
  2,
  15,
  48,
  2,
  2,
  2,
  2,
  2,
  48,
  2,
  2,
  2,
  48,
  2,
  2,
  2,
  2,
  30,
  2,
  2,
  30,
  2,
  48],
 [2, 2, 15, 2, 2, 2, 2, 15, 2, 2, 2, 2, 48, 2, 2, 2, 2, 30, 2, 2, 30, 2, 48],
 [2, 15, 2, 2, 2, 15, 2, 2, 2, 2, 48, 2, 2, 2, 2, 30, 2, 2, 30, 2, 48],
 [2, 15, 2, 2, 15, 48, 2, 2, 2, 2, 48, 2, 2, 2, 48, 2, 2, 2, 2, 2, 48],
 [2

In [26]:
len(mol_encoded_docs)

111216

In [27]:
# pad documents to a max length of 4 words
mol_max_length = 100
mol_padded_docs = pad_sequences(mol_encoded_docs, maxlen=mol_max_length, padding='post')
print(mol_padded_docs)

[[ 2  7  2 ...  0  0  0]
 [ 2  2 15 ...  0  0  0]
 [ 2 15  2 ...  0  0  0]
 ...
 [ 2  2  2 ...  0  0  0]
 [ 2  2  2 ...  0  0  0]
 [ 2  2  2 ...  0  0  0]]


In [28]:
mol_padded_docs.shape

(111216, 100)

In [29]:
mol_padded_docs

array([[ 2,  7,  2, ...,  0,  0,  0],
       [ 2,  2, 15, ...,  0,  0,  0],
       [ 2, 15,  2, ...,  0,  0,  0],
       ...,
       [ 2,  2,  2, ...,  0,  0,  0],
       [ 2,  2,  2, ...,  0,  0,  0],
       [ 2,  2,  2, ...,  0,  0,  0]], dtype=int32)

In [30]:
kkk = np.array(proteins["Sequence"].apply(lambda x:len(x)))

In [31]:
kkk.sort()

In [32]:
(kkk>1500).sum()

55

In [33]:
protein_vocab_size = 8000

In [34]:
protein_encoded_docs = [one_hot(d, protein_vocab_size) for d in protein_corpus]

In [35]:
protein_max_length = 500
protein_padded_docs = pad_sequences(protein_encoded_docs, maxlen=protein_max_length, padding='post')

In [36]:
batch_size = 50

In [37]:
def convert_to_one_hot(y, C):
    y = y.astype(int)
    return np.eye(C)[y.reshape(-1)]

In [38]:
~np.eye(3)[1].astype(bool)

array([ True, False,  True])

In [39]:
class Reduce_Sum(Layer):

    def __init__(self, axis, **kwargs):
        self.axis = axis
        super(Reduce_Sum, self).__init__(**kwargs)


    def call(self, x):
        return K.sum(x, axis = self.axis)
    
    def compute_output_shape(self, input_shape):
        masks = ~np.eye(len(input_shape))[self.axis].astype(bool)
        #print("input_shape", input_shape)
        #print("list(np.array(input_shape)[masks])", list(np.array(input_shape)[masks]))
        return tuple(list(np.array(input_shape)[masks]))


In [40]:
smile_input = Input(shape=(mol_max_length, ), name="input_smiles")
protein_input = Input(shape=(protein_max_length, ), name="input_protein_seqs")
other_input = Input(shape=(21, ), name="input_admet")
smile_embeds = Embedding(mol_vocab_size, 512, input_length=mol_max_length)(smile_input)
protein_embeds = Embedding(protein_vocab_size, 512, input_length=protein_max_length)(protein_input)

smiles1 = Reduce_Sum(axis = 1)(smile_embeds)#Flatten()(smile_embeds)
proteins1 = Reduce_Sum(axis = 1)(protein_embeds)#Flatten()(protein_embeds)

concated = Concatenate()([other_input, smiles1, proteins1])
concated = BatchNormalization()(concated)
dense = Dense(1000, activation='relu')(concated)
dense = BatchNormalization()(dense)
dense = Dense(200, activation='relu')(dense)
dense = BatchNormalization()(dense)
preds = Dense(1)(dense)



In [41]:
smiles1.get_shape().as_list()

[None, 512]

In [42]:
proteins1

<tf.Tensor 'reduce__sum_2/Sum:0' shape=(?, 512) dtype=float32>

In [43]:
smiles1

<tf.Tensor 'reduce__sum_1/Sum:0' shape=(?, 512) dtype=float32>

In [44]:
model = Model(input = [smile_input, protein_input, other_input], output = preds)

/home/stein/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


In [45]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_smiles (InputLayer)       (None, 100)          0                                            
__________________________________________________________________________________________________
input_protein_seqs (InputLayer) (None, 500)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 512)     30720       input_smiles[0][0]               
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 500, 512)     4096000     input_protein_seqs[0][0]         
__________________________________________________________________________________________________
input_adme

In [46]:
model.compile(loss='mse', optimizer='adam', metrics=['mse'])

In [1]:
from sklearn.model_selection import train_test_split

In [6]:
import numpy as np

In [7]:
train_idx, test_idx = train_test_split(np.array(list(range(1000))), test_size = 0.2)

In [8]:
train_idx

array([405, 904, 996, 193,  72, 643, 158, 571, 526, 341, 761, 181, 418,
        64, 200, 355, 138, 696,  38, 477, 970, 849, 430, 633, 496, 112,
       119,  37, 458, 240, 160, 302, 184, 865, 236, 219, 503, 705, 367,
       794, 407, 649, 542, 464, 534, 851, 883, 573,  26,  46,  84,  16,
       345, 500, 278, 936, 802, 510, 898,  92, 490, 396, 963, 815, 331,
       189, 741, 698, 259, 809, 261, 360, 105, 303, 295, 905, 484, 357,
         4, 925, 199, 143, 994, 718, 104, 519, 875,  35, 290, 922, 169,
       948, 937, 338, 780, 128, 316, 695, 583, 995, 157, 751, 634, 383,
       688, 311,  65, 729, 927, 617, 421, 472, 319, 906, 755, 127, 237,
       784,  30, 550, 727, 564, 793, 529, 408, 842, 911, 533, 673, 620,
       535, 555,  99, 175, 438, 253, 821, 115, 161, 693, 999, 466, 568,
        85, 258, 196, 785, 427, 513, 511, 671, 228,  86, 689, 819, 985,
       656, 672, 544,  53, 576, 419, 384, 462, 746, 432, 269, 310, 171,
       941, 973, 605, 771, 919, 957, 712, 244, 699, 212, 300, 30

In [48]:
ped = pd.DataFrame(data=protein_padded_docs, columns = ["pword_"+str(i) for i in range(500)])

In [49]:
med = pd.DataFrame(data=mol_padded_docs, columns=["mword_"+str(i) for i in range(100)])

In [50]:
ped["Protein_ID"] = proteins["Protein_ID"]
med["Molecule_ID"] = mols["Molecule_ID"]

In [51]:
ped

,pword_0,pword_1,pword_2,pword_3,pword_4,pword_5,pword_6,pword_7,pword_8,pword_9,...,pword_491,pword_492,pword_493,pword_494,pword_495,pword_496,pword_497,pword_498,pword_499,Protein_ID
0,4806,2852,3969,5516,168,5344,3304,7014,2310,2837,...,0,0,0,0,0,0,0,0,0,0
1,3359,4952,2632,4013,2256,3794,2648,6888,3595,1667,...,0,0,0,0,0,0,0,0,0,1
2,2343,2883,4468,4528,4673,7888,6696,2832,1354,4255,...,0,0,0,0,0,0,0,0,0,2
3,617,6191,1721,7816,5659,7735,5197,1728,5326,4685,...,0,0,0,0,0,0,0,0,0,3
4,3769,976,6829,7611,7155,4773,6578,4686,6090,6293,...,0,0,0,0,0,0,0,0,0,4
5,3769,890,3014,7611,1463,3350,4658,5504,1409,6293,...,0,0,0,0,0,0,0,0,0,5
6,5873,5627,1682,2315,6975,415,2229,4751,235,6935,...,0,0,0,0,0,0,0,0,0,6
7,617,7858,6248,4928,805,5630,1409,3356,4287,3666,...,0,0,0,0,0,0,0,0,0,7
8,442,4287,4454,2038,2340,1027,1005,3475,5415,7674,...,0,0,0,0,0,0,0,0,0,8
9,7503,4466,6215,1821,5895,7700,2574,5896,6920,7154,...,0,0,0,0,0,0,0,0,0,9


In [52]:
affs.columns = ["Protein_ID","Molecule_ID","Ki"]

In [53]:
affs = affs.merge(med, on="Molecule_ID")

In [54]:
ped

,pword_0,pword_1,pword_2,pword_3,pword_4,pword_5,pword_6,pword_7,pword_8,pword_9,...,pword_491,pword_492,pword_493,pword_494,pword_495,pword_496,pword_497,pword_498,pword_499,Protein_ID
0,4806,2852,3969,5516,168,5344,3304,7014,2310,2837,...,0,0,0,0,0,0,0,0,0,0
1,3359,4952,2632,4013,2256,3794,2648,6888,3595,1667,...,0,0,0,0,0,0,0,0,0,1
2,2343,2883,4468,4528,4673,7888,6696,2832,1354,4255,...,0,0,0,0,0,0,0,0,0,2
3,617,6191,1721,7816,5659,7735,5197,1728,5326,4685,...,0,0,0,0,0,0,0,0,0,3
4,3769,976,6829,7611,7155,4773,6578,4686,6090,6293,...,0,0,0,0,0,0,0,0,0,4
5,3769,890,3014,7611,1463,3350,4658,5504,1409,6293,...,0,0,0,0,0,0,0,0,0,5
6,5873,5627,1682,2315,6975,415,2229,4751,235,6935,...,0,0,0,0,0,0,0,0,0,6
7,617,7858,6248,4928,805,5630,1409,3356,4287,3666,...,0,0,0,0,0,0,0,0,0,7
8,442,4287,4454,2038,2340,1027,1005,3475,5415,7674,...,0,0,0,0,0,0,0,0,0,8
9,7503,4466,6215,1821,5895,7700,2574,5896,6920,7154,...,0,0,0,0,0,0,0,0,0,9


In [55]:
affs = affs.merge(ped, on="Protein_ID")

In [56]:
affs = affs.merge(mols, on="Molecule_ID")

In [57]:
affs

,Protein_ID,Molecule_ID,Ki,mword_0,mword_1,mword_2,mword_3,mword_4,mword_5,mword_6,...,cyp_2d6,tetrahymena_pyriformis_toxicity,Vdd,honey_bee,p_glycoprotein_inhibition,NOAEL,solubility,logS,bbb,smile_corpus
0,523,41360,16.000000,7,2,15,2,44,48,2,...,0.521963,2.390618,7.920029,0.945635,0.802797,-0.789534,-4.149490,-3.774362,0.000809,O = C ( N [ C @ H ] 1 C N 2 C C C 1 C C 2 ) C ...
1,523,91783,11.397940,7,2,48,15,2,2,44,...,0.193774,2.482366,20.607411,0.862675,0.789057,-0.763477,-4.448258,-3.493955,0.000693,O = C 1 N ( C [ C @ H ] 2 C C C C 3 C C C C 1 ...
2,54,91783,11.500313,7,2,48,15,2,2,44,...,0.193774,2.482366,20.607411,0.862675,0.789057,-0.763477,-4.448258,-3.493955,0.000693,O = C 1 N ( C [ C @ H ] 2 C C C C 3 C C C C 1 ...
3,523,64262,11.494850,2,2,2,15,48,2,59,...,0.524928,2.984542,7.733978,0.921582,0.696416,-0.534821,-4.468804,-3.430357,0.022289,C C ( C ) N 1 C 2 C C C C C 2 C ( C C 1 = O ) ...
4,523,66546,11.420217,2,2,48,15,44,2,15,...,0.923047,1.827628,13.431373,0.908751,0.780985,-0.814497,-4.352324,-3.852978,0.012670,C C 1 [ N H ] C [ N H + ] C 1 C N 1 C C N ( C ...
5,523,73107,11.096910,2,2,2,2,48,2,15,...,0.363685,2.652391,6.375209,0.933640,0.440314,-0.903642,-4.215707,-3.414495,0.002460,C C C C 1 C ( N C 2 C C C C C 2 C 1 C ( = O ) ...
6,523,59485,11.000000,15,2,48,2,59,2,2,...,0.781942,2.186273,6.894355,0.937075,0.888919,-0.805799,-3.731553,-3.775524,0.073976,N C 1 C 2 C C O C 2 C ( C C 1 C L ) C ( = O ) ...
7,252,59485,8.769551,15,2,48,2,59,2,2,...,0.781942,2.186273,6.894355,0.937075,0.888919,-0.805799,-3.731553,-3.775524,0.073976,N C 1 C 2 C C O C 2 C ( C C 1 C L ) C ( = O ) ...
8,523,73110,10.958608,2,2,2,15,2,2,2,...,0.322941,3.041183,8.900435,0.934778,0.363383,-1.017184,-4.752351,-3.925998,0.002302,C C C N ( C C C ) C ( = O ) C 1 C ( C ) C ( N ...
9,523,59486,7.602060,2,2,2,2,15,48,2,...,0.375820,2.649108,6.082622,0.883057,0.556858,-0.927017,-4.227768,-4.178930,0.010825,C C C C N 1 C C C ( C O C ( = O ) C 2 C C ( C ...


In [58]:
affs

,Protein_ID,Molecule_ID,Ki,mword_0,mword_1,mword_2,mword_3,mword_4,mword_5,mword_6,...,cyp_2d6,tetrahymena_pyriformis_toxicity,Vdd,honey_bee,p_glycoprotein_inhibition,NOAEL,solubility,logS,bbb,smile_corpus
0,523,41360,16.000000,7,2,15,2,44,48,2,...,0.521963,2.390618,7.920029,0.945635,0.802797,-0.789534,-4.149490,-3.774362,0.000809,O = C ( N [ C @ H ] 1 C N 2 C C C 1 C C 2 ) C ...
1,523,91783,11.397940,7,2,48,15,2,2,44,...,0.193774,2.482366,20.607411,0.862675,0.789057,-0.763477,-4.448258,-3.493955,0.000693,O = C 1 N ( C [ C @ H ] 2 C C C C 3 C C C C 1 ...
2,54,91783,11.500313,7,2,48,15,2,2,44,...,0.193774,2.482366,20.607411,0.862675,0.789057,-0.763477,-4.448258,-3.493955,0.000693,O = C 1 N ( C [ C @ H ] 2 C C C C 3 C C C C 1 ...
3,523,64262,11.494850,2,2,2,15,48,2,59,...,0.524928,2.984542,7.733978,0.921582,0.696416,-0.534821,-4.468804,-3.430357,0.022289,C C ( C ) N 1 C 2 C C C C C 2 C ( C C 1 = O ) ...
4,523,66546,11.420217,2,2,48,15,44,2,15,...,0.923047,1.827628,13.431373,0.908751,0.780985,-0.814497,-4.352324,-3.852978,0.012670,C C 1 [ N H ] C [ N H + ] C 1 C N 1 C C N ( C ...
5,523,73107,11.096910,2,2,2,2,48,2,15,...,0.363685,2.652391,6.375209,0.933640,0.440314,-0.903642,-4.215707,-3.414495,0.002460,C C C C 1 C ( N C 2 C C C C C 2 C 1 C ( = O ) ...
6,523,59485,11.000000,15,2,48,2,59,2,2,...,0.781942,2.186273,6.894355,0.937075,0.888919,-0.805799,-3.731553,-3.775524,0.073976,N C 1 C 2 C C O C 2 C ( C C 1 C L ) C ( = O ) ...
7,252,59485,8.769551,15,2,48,2,59,2,2,...,0.781942,2.186273,6.894355,0.937075,0.888919,-0.805799,-3.731553,-3.775524,0.073976,N C 1 C 2 C C O C 2 C ( C C 1 C L ) C ( = O ) ...
8,523,73110,10.958608,2,2,2,15,2,2,2,...,0.322941,3.041183,8.900435,0.934778,0.363383,-1.017184,-4.752351,-3.925998,0.002302,C C C N ( C C C ) C ( = O ) C 1 C ( C ) C ( N ...
9,523,59486,7.602060,2,2,2,2,15,48,2,...,0.375820,2.649108,6.082622,0.883057,0.556858,-0.927017,-4.227768,-4.178930,0.010825,C C C C N 1 C C C ( C O C ( = O ) C 2 C C ( C ...


In [59]:
affs.reset_index(drop=True, inplace=True)

In [60]:
from sklearn.model_selection import train_test_split

In [61]:
training_proteins, test_proteins = train_test_split(np.unique(affs.Protein_ID.values), test_size = 0.2)

In [62]:
trainin_data = affs[affs.Protein_ID.isin(training_proteins)]

In [63]:
test_data = affs[affs.Protein_ID.isin(test_proteins)]

In [64]:
trainin_data

,Protein_ID,Molecule_ID,Ki,mword_0,mword_1,mword_2,mword_3,mword_4,mword_5,mword_6,...,cyp_2d6,tetrahymena_pyriformis_toxicity,Vdd,honey_bee,p_glycoprotein_inhibition,NOAEL,solubility,logS,bbb,smile_corpus
0,523,41360,16.000000,7,2,15,2,44,48,2,...,0.521963,2.390618,7.920029,0.945635,0.802797,-0.789534,-4.149490,-3.774362,0.000809,O = C ( N [ C @ H ] 1 C N 2 C C C 1 C C 2 ) C ...
1,523,91783,11.397940,7,2,48,15,2,2,44,...,0.193774,2.482366,20.607411,0.862675,0.789057,-0.763477,-4.448258,-3.493955,0.000693,O = C 1 N ( C [ C @ H ] 2 C C C C 3 C C C C 1 ...
3,523,64262,11.494850,2,2,2,15,48,2,59,...,0.524928,2.984542,7.733978,0.921582,0.696416,-0.534821,-4.468804,-3.430357,0.022289,C C ( C ) N 1 C 2 C C C C C 2 C ( C C 1 = O ) ...
4,523,66546,11.420217,2,2,48,15,44,2,15,...,0.923047,1.827628,13.431373,0.908751,0.780985,-0.814497,-4.352324,-3.852978,0.012670,C C 1 [ N H ] C [ N H + ] C 1 C N 1 C C N ( C ...
5,523,73107,11.096910,2,2,2,2,48,2,15,...,0.363685,2.652391,6.375209,0.933640,0.440314,-0.903642,-4.215707,-3.414495,0.002460,C C C C 1 C ( N C 2 C C C C C 2 C 1 C ( = O ) ...
6,523,59485,11.000000,15,2,48,2,59,2,2,...,0.781942,2.186273,6.894355,0.937075,0.888919,-0.805799,-3.731553,-3.775524,0.073976,N C 1 C 2 C C O C 2 C ( C C 1 C L ) C ( = O ) ...
7,252,59485,8.769551,15,2,48,2,59,2,2,...,0.781942,2.186273,6.894355,0.937075,0.888919,-0.805799,-3.731553,-3.775524,0.073976,N C 1 C 2 C C O C 2 C ( C C 1 C L ) C ( = O ) ...
8,523,73110,10.958608,2,2,2,15,2,2,2,...,0.322941,3.041183,8.900435,0.934778,0.363383,-1.017184,-4.752351,-3.925998,0.002302,C C C N ( C C C ) C ( = O ) C 1 C ( C ) C ( N ...
9,523,59486,7.602060,2,2,2,2,15,48,2,...,0.375820,2.649108,6.082622,0.883057,0.556858,-0.927017,-4.227768,-4.178930,0.010825,C C C C N 1 C C C ( C O C ( = O ) C 2 C C ( C ...
10,252,59486,10.886057,2,2,2,2,15,48,2,...,0.375820,2.649108,6.082622,0.883057,0.556858,-0.927017,-4.227768,-4.178930,0.010825,C C C C N 1 C C C ( C O C ( = O ) C 2 C C ( C ...


In [65]:
test_data

,Protein_ID,Molecule_ID,Ki,mword_0,mword_1,mword_2,mword_3,mword_4,mword_5,mword_6,...,cyp_2d6,tetrahymena_pyriformis_toxicity,Vdd,honey_bee,p_glycoprotein_inhibition,NOAEL,solubility,logS,bbb,smile_corpus
2,54,91783,11.500313,7,2,48,15,2,2,44,...,0.193774,2.482366,20.607411,0.862675,0.789057,-0.763477,-4.448258,-3.493955,0.000693,O = C 1 N ( C [ C @ H ] 2 C C C C 3 C C C C 1 ...
50,54,13775,10.443698,2,2,48,2,2,2,2,...,0.626147,2.457895,7.487914,0.884866,0.460443,-1.026208,-3.224381,-3.166644,0.000592,C ( C 1 C C C C C 1 ) N 1 C ( N C 2 C C C C C ...
134,54,39774,10.000000,2,2,15,48,2,59,2,...,0.561130,2.266802,6.960585,0.925425,0.821042,-0.768117,-3.803277,-2.496460,0.031157,C C N 1 C 2 C C C C C 2 N ( C ( = O ) N C 2 C ...
135,494,39774,8.886057,2,2,15,48,2,59,2,...,0.561130,2.266802,6.960585,0.925425,0.821042,-0.768117,-3.803277,-2.496460,0.031157,C C N 1 C 2 C C C C C 2 N ( C ( = O ) N C 2 C ...
139,54,59498,10.000000,2,30,2,48,2,2,59,...,0.679766,2.132925,5.803199,0.944650,0.887317,-0.756019,-3.768101,-3.662726,0.010465,C L C 1 C C 2 C C O C 2 C ( C 1 ) C ( = O ) N ...
191,365,63243,7.000000,2,2,48,15,2,15,44,...,0.947085,1.604071,7.822136,0.680700,0.841083,-1.144965,-3.624846,-3.447783,0.003851,C C 1 N C [ N H ] C 1 C C 1 N C ( C S 1 ) - C ...
193,43,63243,7.000000,2,2,48,15,2,15,44,...,0.947085,1.604071,7.822136,0.680700,0.841083,-1.144965,-3.624846,-3.447783,0.003851,C C 1 N C [ N H ] C 1 C C 1 N C ( C S 1 ) - C ...
196,54,63243,9.744727,2,2,48,15,2,15,44,...,0.947085,1.604071,7.822136,0.680700,0.841083,-1.144965,-3.624846,-3.447783,0.003851,C C 1 N C [ N H ] C 1 C C 1 N C ( C S 1 ) - C ...
206,54,58841,9.795880,2,2,44,48,2,2,2,...,0.353150,2.627312,7.271181,0.872787,0.514967,-0.840570,-3.955871,-3.273563,0.011569,C [ C @ @ H ] 1 C C C N 1 C C C O C 1 C C C ( ...
213,398,87593,9.795880,2,2,48,2,2,2,21,...,0.176967,2.315991,9.079201,0.943405,0.595564,-0.786868,-4.524537,-4.473687,0.000440,C C 1 C C C ( S C 2 C C C C C 2 N 2 C C N C C ...


In [66]:
np.unique(affs.Protein_ID.values)

array([   0,    1,    2, ..., 2064, 2065, 2066])

In [67]:
smile_docs_feats = ["mword_"+str(i) for i in range(100)]

In [68]:
pdocs_feats = ['pword_'+str(i) for i in range(500)]

In [69]:
admet_feats = [i for i in list(mols.columns) if i not in ['smiles','name','Molecule_ID']]

In [70]:
admet_feats.remove("smile_corpus")

In [71]:
admet_feats

['cyp_2c9',
 'half-life',
 'ames_toxicity',
 'logBB',
 'cyp_3a4',
 'cell_permeability',
 'logP',
 'fathead_minnow_toxicity',
 'renal_organic_cation_transporter',
 'CLtotal',
 'hia',
 'biodegradation',
 'cyp_2d6',
 'tetrahymena_pyriformis_toxicity',
 'Vdd',
 'honey_bee',
 'p_glycoprotein_inhibition',
 'NOAEL',
 'solubility',
 'logS',
 'bbb']

In [12]:
dd = np.zeros([3,4]).astype('int8')

In [15]:
dd[0] = [1,2,3,4]

In [16]:
dd

array([[1, 2, 3, 4],
       [0, 0, 0, 0],
       [0, 0, 0, 0]], dtype=int8)

In [72]:
hists = model.fit([trainin_data[smile_docs_feats], trainin_data[pdocs_feats], trainin_data[admet_feats]], 
                  trainin_data.Ki, batch_size=100, epochs=10, 
                  validation_data=([test_data[smile_docs_feats], test_data[pdocs_feats], test_data[admet_feats]], test_data.Ki),
                 )

Train on 166760 samples, validate on 39707 samples
Epoch 1/10
166760/166760 [==============================] - 194s 1ms/step - loss: 4.7303 - mean_squared_error: 4.7303 - val_loss: 2.7503 - val_mean_squared_error: 2.7503
Epoch 2/10
166760/166760 [==============================] - 198s 1ms/step - loss: 1.4816 - mean_squared_error: 1.4816 - val_loss: 2.4585 - val_mean_squared_error: 2.4585
Epoch 3/10
166760/166760 [==============================] - 190s 1ms/step - loss: 1.3739 - mean_squared_error: 1.3739 - val_loss: 2.7038 - val_mean_squared_error: 2.7038
Epoch 4/10
166760/166760 [==============================] - 190s 1ms/step - loss: 1.2934 - mean_squared_error: 1.2934 - val_loss: 2.5262 - val_mean_squared_error: 2.5262
Epoch 5/10
166760/166760 [==============================] - 193s 1ms/step - loss: 1.2227 - mean_squared_error: 1.2227 - val_loss: 2.3934 - val_mean_squared_error: 2.3934
Epoch 6/10
166760/166760 [==============================] - 192s 1ms/step - loss: 1.1657 - mean_squ

In [70]:
all_words = ''
for i in range(len(protein_corpus)):
    all_words+= " "+protein_corpus[i]

In [72]:
word_sets = all_words.split(' ')

In [76]:
len(np.unique(word_sets))

7996